In [1]:
import keras as k
import keras.backend as b
import keras.layers as l
import numpy as np
import sys
import theano as t
import keras.initializations as i
sys.path.append('../src/mane/')

Using Theano backend.


In [5]:
import graph as g
reload(g)
fb = g.graph_from_pickle('../src/mane/data/egonets.graph')

In [6]:
awesome = fb.gen_contrast()

In [7]:
next(awesome)

({'class': array([2600, 2600, 2546, 2287, 2287, 1979, 2232, 2624, 2543, 2546, 2624,
         2624, 2546, 2414, 2546, 2414, 2546, 2164, 2625, 2546, 2232, 2655,
         2090, 2543, 2655, 2140, 2140, 2543, 1979, 2287, 2546, 2140, 2140,
         2543, 2625, 2428, 2414, 2414, 2655, 2232, 2232, 2150, 2543, 1979,
         2287, 2287, 2655, 2150, 1979, 2655, 2543, 2164, 2164, 2624, 2546,
         2232, 2543, 1979, 2287, 2090, 2090, 2655, 2546, 2655, 2655, 2140,
         2414, 2232, 2428, 2229, 2624, 2546, 2414, 2546, 2546, 1979, 1979,
         2140, 2090, 2543, 2624, 2287, 2232, 2543, 2546, 2625, 2363, 2624,
         2655, 2543, 2543, 2655, 1979, 2232, 1979, 2232, 2414, 2140, 2546,
         2414, 2543, 1979, 2229, 2553, 2624, 2414, 2287, 2090, 2414, 2287,
         2655, 2655, 2090, 2287, 2150, 2655, 2624, 2090, 1979, 2550, 2553,
         2655, 2287, 2414, 2655, 2140, 2232, 2546, 2140, 2655, 2655, 2546,
         2140, 1979, 2655, 2655, 2553, 2482, 2624, 2090, 2150, 2150, 2414,
         2655, 2

In [8]:
la = next(awesome)

In [10]:
len(la[0]['class'])

340